In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv('clean_final.csv', index_col=0)

In [3]:
df

,customer_no,location,cust_id,time,day,section_order,revenue
timestamp,,,,,,,
2019-09-02 07:03:00,1,dairy,m_1,07:03:00,Monday,first,5
2019-09-02 07:03:00,2,dairy,m_2,07:03:00,Monday,first,5
2019-09-02 07:04:00,1,dairy,m_1,07:03:00,Monday,first,5
2019-09-02 07:04:00,2,dairy,m_2,07:03:00,Monday,first,5
2019-09-02 07:04:00,3,dairy,m_3,07:04:00,Monday,first,5
...,...,...,...,...,...,...,...
2019-09-06 21:50:00,1509,drinks,fr_1509,21:50:00,Friday,first,6
2019-09-06 21:50:00,1510,spices,fr_1510,21:50:00,Friday,first,3
2019-09-06 21:51:00,1506,checkout,fr_1506,21:51:00,Friday,checkout,0


In [6]:
df['next_location'] = df['location'].shift(-1)
df

,customer_no,location,cust_id,time,day,section_order,revenue,next_location
timestamp,,,,,,,,
2019-09-02 07:03:00,1,dairy,m_1,07:03:00,Monday,first,5,dairy
2019-09-02 07:03:00,2,dairy,m_2,07:03:00,Monday,first,5,dairy
2019-09-02 07:04:00,1,dairy,m_1,07:03:00,Monday,first,5,dairy
2019-09-02 07:04:00,2,dairy,m_2,07:03:00,Monday,first,5,dairy
2019-09-02 07:04:00,3,dairy,m_3,07:04:00,Monday,first,5,dairy
...,...,...,...,...,...,...,...,...
2019-09-06 21:50:00,1509,drinks,fr_1509,21:50:00,Friday,first,6,spices
2019-09-06 21:50:00,1510,spices,fr_1510,21:50:00,Friday,first,3,checkout
2019-09-06 21:51:00,1506,checkout,fr_1506,21:51:00,Friday,checkout,0,checkout


In [7]:
df.groupby('location')['next_location'].value_counts().unstack()

next_location,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,1133,2420,1326,1681,865
dairy,2455,5869,3313,4126,2016
drinks,1395,3172,1780,2264,1115
fruit,1616,4246,2210,3136,1504
spices,827,2071,1097,1505,778


In [8]:
df_1 = df[df['location'] != 'checkout'].copy()

In [9]:
trans_without_checkout= pd.crosstab(df_1['location'], df_1['next_location'], normalize=0)

In [10]:
trans_without_checkout

next_location,checkout,dairy,drinks,fruit,spices
location,,,,,
dairy,0.138084,0.330109,0.186343,0.232072,0.113392
drinks,0.143430,0.326136,0.183015,0.232778,0.114641
fruit,0.127124,0.334015,0.173851,0.246696,0.118313
spices,0.131730,0.329882,0.174737,0.239726,0.123925


In [11]:
prob = np.array([1.0,0.0,0.0,0.0,0.0,
                0.138084,0.330109,0.186343,0.232072,0.113392,
                0.143430,0.326136,0.183015,0.232778,0.114641,
                0.127124,0.334015,0.173851,0.246696,0.118313,
                0.131730,0.329882,0.174737,0.239726,0.123925
                ]).reshape(5,5)
trans_matrix = pd.DataFrame(prob,columns=['checkout','dairy','drinks','fruit','spices'],
                index=['checkout','dairy','drinks','fruit','spices'])

In [12]:
trans_matrix

,checkout,dairy,drinks,fruit,spices
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.138084,0.330109,0.186343,0.232072,0.113392
drinks,0.143430,0.326136,0.183015,0.232778,0.114641
fruit,0.127124,0.334015,0.173851,0.246696,0.118313
spices,0.131730,0.329882,0.174737,0.239726,0.123925


In [13]:
entrance = np.identity(1)

# Add a random drift term.  We can guarantee that the diagonal terms
#     will be larger by specifying a `high` parameter that is < 1.
# How much larger depends on that term.  Here, it is 0.25.
entrance = entrance + np.random.uniform(low=0. , size=(1, 4))

# Lastly, divide by row-wise sum to normalize to 1.
entrance = entrance / entrance.sum(axis=1, keepdims=1)



In [63]:
entrance_number = (round(
    df[df["section_order"] == "first"]
    .groupby(["time"])[["cust_id"]]
    .count()
    / 5, 0)
)
entrance_number

,cust_id
time,
07:00:00,5.0
07:01:00,7.0
07:02:00,4.0
07:03:00,8.0
07:04:00,6.0
...,...
21:46:00,4.0
21:47:00,1.0
21:48:00,3.0


In [61]:
def get_customer_number(time):
    for row, number in zip(entrance_number.index, entrance_number['cust_id']):
        if row == time:
            return(number)
        else:
            return('Supermarket is closed')      


In [62]:
get_customer_number('22:03:00')

'Supermarket is closed'